In [2]:
%pip install pandas 
%pip install numpy

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
import numpy as np



In [4]:
stavbe2021 = pd.read_csv(r"clean_GURS\new_prodaja2021.csv", sep=',', encoding='utf-8')
stavbe2021.sample(10)

,ime_ko,naselje,letnica_izgradnje,m2_bruto,m2_neto,LETO,cena,novogradnja
2658,SPODNJA ŠIŠKA,LJUBLJANA,1946,37.1,32.0,2021,155894.0,NaN
1567,POLJANSKO PREDMESTJE,LJUBLJANA,1990,27.6,27.6,2021,85000.0,NaN
722,ZGORNJA ŠIŠKA,LJUBLJANA,1972,61.5,57.4,2021,200000.0,NaN
1306,GLINCE,LJUBLJANA,2008,55.1,46.4,2021,200000.0,NaN
2154,GLINCE,LJUBLJANA,2008,188.2,147.2,2021,360000.0,NaN
603,ČRNUČE,LJUBLJANA,1973,94.1,90.6,2021,234000.0,NaN
2194,TABOR,LJUBLJANA,1928,60.4,44.6,2021,230000.0,NaN
2604,ZGORNJA ŠIŠKA,LJUBLJANA,2009,77.6,74.2,2021,265000.0,NaN
667,TRNOVSKO PREDMESTJE,LJUBLJANA,2017,79.0,56.1,2021,282743.0,1.0
422,MOSTE,LJUBLJANA,1984,25.2,23.5,2021,37500.0,NaN


In [5]:
stavbe2021.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2927 entries, 0 to 2926
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   ime_ko             2927 non-null   object 
 1   naselje            2902 non-null   object 
 2   letnica_izgradnje  2927 non-null   int64  
 3   m2_bruto           2927 non-null   float64
 4   m2_neto            2927 non-null   float64
 5   LETO               2927 non-null   int64  
 6   cena               2926 non-null   float64
 7   novogradnja        351 non-null    float64
dtypes: float64(4), int64(2), object(2)
memory usage: 183.1+ KB


In [6]:
#preparing the year column for Power BI
stavbe2021['LETO'] = pd.to_datetime("2021-01-01")

In [7]:

# creating a boolean column 'NOVOGRADNJA_1' based on 'NOVOGRADNJA'
stavbe2021['novogradnja_1'] = stavbe2021['novogradnja'] == 1



In [8]:
stavbe2021[['novogradnja', 'novogradnja_1']].sample(10)

,novogradnja,novogradnja_1
2716,1.0,True
1445,1.0,True
1682,NaN,False
2382,NaN,False
69,NaN,False
1486,NaN,False
1043,NaN,False
2893,NaN,False
733,NaN,False
1631,NaN,False


In [9]:
#Printing the number of rows with missing values in 'm2_neto' column
stavbe2021[stavbe2021['m2_neto'].isna()]

,ime_ko,naselje,letnica_izgradnje,m2_bruto,m2_neto,LETO,cena,novogradnja,novogradnja_1


In [10]:
stavbe2021[stavbe2021['m2_neto'] == 0].count()

ime_ko               49
naselje              37
letnica_izgradnje    49
m2_bruto             49
m2_neto              49
LETO                 49
cena                 49
novogradnja          24
novogradnja_1        49
dtype: int64

In [11]:
stavbe2021.loc[stavbe2021['m2_neto'] == 0, 'm2_neto'] = np.nan
stavbe2021.fillna({'m2_neto': stavbe2021['m2_bruto']}, inplace=True)

In [13]:
def oceni_sobnost(m2):
    if pd.isna(m2):
        return None
    if m2 < 35:
        return 1
    elif m2 < 50:
        return 2
    elif m2 < 70:
        return 3
    elif m2 < 90:
        return 4
    elif m2 < 110:
        return 5
    else:
        return 6

In [14]:
stavbe2021['sobnost_ocenjena'] = stavbe2021['m2_neto'].apply(oceni_sobnost)

In [15]:
stavbe2021.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2927 entries, 0 to 2926
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   ime_ko             2927 non-null   object        
 1   naselje            2902 non-null   object        
 2   letnica_izgradnje  2927 non-null   int64         
 3   m2_bruto           2927 non-null   float64       
 4   m2_neto            2927 non-null   float64       
 5   LETO               2927 non-null   datetime64[ns]
 6   cena               2926 non-null   float64       
 7   novogradnja        351 non-null    float64       
 8   novogradnja_1      2927 non-null   bool          
 9   sobnost_ocenjena   2927 non-null   int64         
dtypes: bool(1), datetime64[ns](1), float64(4), int64(2), object(2)
memory usage: 208.8+ KB


In [16]:
# Droping the original 'NOVOGRADNJA' column, as we now have 'NOVOGRADNJA_1' which has the same information in a boolean format.
stavbe2021.drop(columns=['novogradnja'], inplace=True)

In [17]:
#Printing the number of rows with missing values in 'cena' column
stavbe2021[stavbe2021['cena'].isna()]

,ime_ko,naselje,letnica_izgradnje,m2_bruto,m2_neto,LETO,cena,novogradnja_1,sobnost_ocenjena
484,BRINJE I,LJUBLJANA,1964,75.7,65.7,2021-01-01,NaN,False,3


In [18]:
stavbe2021.drop(index=484, inplace=True)  # Dropping the row with index 484 as it has a missing 'cena' value

In [19]:
# Uniforming the column names to lowercase
stavbe2021.columns = [col.lower() for col in stavbe2021.columns]

In [20]:
#uniforming the all values in all the columns to lowercase
for col in stavbe2021.columns:
    if stavbe2021[col].dtype == 'object':
        stavbe2021[col] = stavbe2021[col].str.lower()

In [22]:
# checking all the empty values in the 'naselje' column
stavbe2021[stavbe2021['naselje'].isna()]

,ime_ko,naselje,letnica_izgradnje,m2_bruto,m2_neto,leto,cena,sobnost_ocenjena
207,vižmarje,NaN,2020,145.6,0.0,2021,200000.0,1
261,vižmarje,NaN,2020,145.6,0.0,2021,207500.0,1
357,vižmarje,NaN,2020,145.6,0.0,2021,292000.0,1
427,udmat,NaN,1975,58.8,58.8,2021,160000.0,3
517,kašelj,NaN,2021,182.0,0.0,2021,190000.0,1
705,tacen,NaN,2013,124.1,102.3,2021,530000.0,5
775,grič,NaN,2019,191.7,179.6,2021,250000.0,6
776,vič,NaN,2000,149.4,112.6,2021,239000.0,6
1190,gradišče ii,NaN,1930,108.5,86.8,2021,158000.0,4
1233,vižmarje,NaN,2020,145.6,0.0,2021,200000.0,1


In [22]:
# changing all the empty values in the 'naselje' column to 'ljubljana'
stavbe2021.fillna({'naselje': 'ljubljana'}, inplace=True)

In [23]:
stavbe2021.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2926 entries, 0 to 2926
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   ime_ko             2926 non-null   object        
 1   naselje            2926 non-null   object        
 2   letnica_izgradnje  2926 non-null   int64         
 3   m2_bruto           2926 non-null   float64       
 4   m2_neto            2926 non-null   float64       
 5   leto               2926 non-null   datetime64[ns]
 6   cena               2926 non-null   float64       
 7   novogradnja_1      2926 non-null   bool          
 8   sobnost_ocenjena   2926 non-null   int64         
dtypes: bool(1), datetime64[ns](1), float64(3), int64(2), object(2)
memory usage: 208.6+ KB


In [24]:
stavbe2021.to_csv(r"clean_GURS\stavbe2021_cleaned.csv", index=False, encoding='utf-8')